<a href="https://colab.research.google.com/github/johanhoffman/DD2365_VT22/blob/main/template-report-Stokes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **The Stokes equations**
**Anton Scotte**

# **Abstract**

In this report, a solution method based on Finite Element discretizations of the weak form of the Stokes equations are implemented in the python package FEniCS. The Stokes equations are used to model viscous flow. Taylor-Hood elements as well as equal order elements for velocity and pressure are used for the simulation and adaptive mesh refinement has been implemented as well. The velocity and pressure solutions has been investigated with respect to stability and stability is verified for the Taylor-Hood elements where the Inf-Suf condition is valid. The Inf-sup condition is however, not fullfilled, with equal order elements as the results show. 

# **About the code**
This program code has been modified from a Template code: obtained at (https://github.com/johanhoffman/DD2365_VT21), for the Stokes equations written originally by Johan Hoffman. The changes that has been made from the original template consists of changes in the mesh setup, adaptive mesh refinement, boundary conditions and the implementation of the equal order elements for comparison to the Taylor hood elements. 

In [ ]:
"""This program is based on an example file for the course"""
"""DD2365 Advanced Computation in Fluid Mechanics, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

# **Set up environment:** 
This code should be run in the Google Colab environment and is setup as follows:

In [ ]:
# Load neccessary modules.
from google.colab import files

import numpy as np
import time

# Install FEniCS (this may take a long time)
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
    
from dolfin import *; from mshr import *

import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt

# **Introduction**

The Stokes equations take the form

$\nabla p -\Delta u = f,\quad \nabla \cdot u=0$

together with suitable boundary conditions.

Here we present a FEniCS implementation of a mixed finite element method to solve the Stokes equations in 2D. The solution is visualized using FEniCS plotting functions, and is also exported as pvd-files which can be visualized in Paraview.

To derive the weak form of the equations, multiply the momentum equation by $v\in V$ and the continuity equation by $q\in Q$, and then integrate over the domain $\Omega$ and use Green's formula
$
(\nabla p -\Delta u,v) = -(p,\nabla \cdot v) + (\nabla u, \nabla v) 
+ <pn - \nabla u\cdot n, v>_{\Gamma}
$

We seek a finite element approximation $(u,p)\in V\times Q$ such that 

$- (p,\nabla \cdot v) + (\nabla u,\nabla v) + (\nabla \cdot u, q) 
+ <pn - \nabla u\cdot n, v>_{\partial \Omega} = (f,v)$

for all test functions $(v,q) \in V\times Q$. 

$
(v,w) = \int_{\Omega} v\cdot w ~dx, \quad 
<v,w>_{\partial \Omega} = \int_{\partial \Omega} v\cdot w~ds
$

We divide the boundary into $\partial \Omega=\Gamma_D \cup \Gamma_N$, with the different boundary conditions

$
u = g_D,\quad x\in \Gamma_D,
$

$
-\nu \nabla u\cdot n + pn = g_N, \quad x\in \Gamma_N,
$

For $x\in \Gamma_D$ the test function $v=0$. Hence, with $g_N=0$ the boundary term is zero. 

The equations can be expressed in residual form

$r(u,p;v,q) = - (p,\nabla \cdot v) + (\nabla u,\nabla v) + (\nabla \cdot u, q) - (f,v)$

We use inf-sup stable Taylor-Hood approximation spaces, 
and to implement the velocity boundary conditions we use a penalty formulation, with a penalty parameter $\gamma = C/h$, with $C>0$ a constant and $h$ the local mesh size. At outflow we use a "do nothing" stress free boundary condition. 


# **Method**

**Define domain:**
The computational domain has been modified to be a rectangular channel with height: 2 units and length: 4 units with a circular hole with radius: 0.2 units at the center of the channel. The resolution is set to be a uniform mesh size of h=1/32. Here we have the option to locally refine the mesh, and this refinement is done on the mesh from the center of the circle with a radius of 1 around the hole. A non-refined mesh as well as the refined mesh are presented for comparison. The variable "mesh" is the non-refined mesh and the variable "mesh2" is refined one time. For further use in the code implementation "mesh2" will be used.

**Non-refined mesh**

In [ ]:
# Define rectangular domain 
L = 4
H = 2

# Generate mesh
resolution = 32
mesh = generate_mesh(Rectangle(Point(0.0,0.0), Point(L,H)) - Circle(Point(0.5*L,0.5*H),0.2), resolution)

plt.figure()
plot(mesh)
plt.show()

**Refined mesh**

In [ ]:
mesh2 = generate_mesh(Rectangle(Point(0.0,0.0), Point(L,H)) - Circle(Point(0.5*L,0.5*H),0.2), resolution)

# Local mesh refinement (specified by a cell marker)
no_levels = 1
for i in range(0,no_levels):
  cell_marker = MeshFunction("bool", mesh2, mesh.topology().dim())
  for cell in cells(mesh2):
    cell_marker[cell] = False
    p = cell.midpoint()
    if p.distance(Point(0.5*L, 0.5*H)) < 1.0:
        cell_marker[cell] = True
  mesh2 = refine(mesh2, cell_marker)


plt.figure()
plot(mesh2)
plt.show()

**Finite element approximation spaces:** The finite element approximation spaces are created for velocity and pressure on the refined mesh, (in the code this mesh is named "mesh2"). These are created with Continuous Galerkin ("CG") elements. First this is tested for Taylor hood elements, created by a 2nd order element VE1 for velocity and a first order element QE1 for pressure. The first order elements results in function spaces that are approximated by piecewise linear polynomials and for the the 2nd order elements the corresponding function space is approximated by piecewise quadratic polynomials. 

The mixed finite element space is then WE1. Then equal order CG elements for pressure and velocity are also implemented both of order 1 for velocity and for pressure. This is in the code defined as VE2, QE2 and WE2. 

In [ ]:
# Generate mixed finite element spaces (for velocity and pressure)
#https://fenicsproject.org/olddocs/dolfin/1.3.0/python/demo/documented/stokes-taylor-hood/python/documentation.html
#https://fenicsproject.org/olddocs/dolfin/1.3.0/python/programmers-reference/functions/functionspace/VectorFunctionSpace.html#dolfin.functions.functionspace.VectorFunctionSpace
#https://fenicsproject.org/olddocs/dolfin/1.3.0/python/programmers-reference/cpp/function/FunctionSpace.html#dolfin.cpp.function.FunctionSpace
#Equal order, eg. set both VE and QE to order 1 or both VE and QE to order 2
#Instability occurs in pressure solution when order is equal but velocity looks fine
#when order is higher for velocity instability in pressure does not occur
#when order is higher for pressure than it is for velocity the Newton Solver does not converge

# Generate mixed finite element spaces (for velocity and pressure)
VE1 = VectorElement("CG", mesh2.ufl_cell(), 2)
QE1 = FiniteElement("CG", mesh2.ufl_cell(), 1)
WE1 = VE1 * QE1

W1 = FunctionSpace(mesh2, WE1)
V1 = FunctionSpace(mesh2, VE1)
Q1 = FunctionSpace(mesh2, QE1)

# Define trial and test functions
w1 = Function(W1)
(u1, p1) = (as_vector((w1[0],w1[1])), w1[2])
(v1, q1) = TestFunctions(W1) 

VE2 = VectorElement("CG", mesh2.ufl_cell(), 1)
QE2 = FiniteElement("CG", mesh2.ufl_cell(), 1)
WE2 = VE2 * QE2

W2 = FunctionSpace(mesh2, WE2)
V2 = FunctionSpace(mesh2, VE2)
Q2 = FunctionSpace(mesh2, QE2)

# Define trial and test functions
w2 = Function(W2)
(u2, p2) = (as_vector((w2[0],w2[1])), w2[2])
(v2, q2) = TestFunctions(W2) 

**Define boundary conditions:** At the walls (top and bottom), the boundary conditions are set to have no-slip BC, that is, Dirichlet BC for the velocity is set to zero. The inflow profile is a parabolic profile with a maximum velocity at $y=H/2$ where $H$ is the height of the channel. In the programme code the BC are specified twice to generate results for both the Taylor-Hood elements as well as for the equal order elements. 

In [ ]:
# Examples of inflow and outflow conditions
XMIN = 0.0; XMAX = L
YMIN = 0.0; YMAX = H
uin1 = Expression(("4*(x[1]*(YMAX-x[1]))/(YMAX*YMAX)", "0."), YMAX=YMAX, element = V1.ufl_element())
#with equal order 
uin2 = Expression(("4*(x[1]*(YMAX-x[1]))/(YMAX*YMAX)", "0."), YMAX=YMAX, element = V2.ufl_element()) 

#pout = 0.0

# Inflow boundary (ib), outflow boundary (ob) and wall boundary (wb)
ib1 = Expression("near(x[0],XMIN) ? 1. : 0.", XMIN=XMIN, element = Q1.ufl_element())
ob1 = Expression("near(x[0],XMAX) ? 1. : 0.", XMAX=XMAX, element = Q1.ufl_element()) 
wb1 = Expression("x[0] > XMIN + DOLFIN_EPS && x[0] < XMAX - DOLFIN_EPS ? 1. : 0.", XMIN=XMIN, XMAX=XMAX, element = Q1.ufl_element())

# Inflow boundary (ib), outflow boundary (ob) and wall boundary (wb)
ib2 = Expression("near(x[0],XMIN) ? 1. : 0.", XMIN=XMIN, element = Q2.ufl_element())
ob2 = Expression("near(x[0],XMAX) ? 1. : 0.", XMAX=XMAX, element = Q2.ufl_element()) 
wb2 = Expression("x[0] > XMIN + DOLFIN_EPS && x[0] < XMAX - DOLFIN_EPS ? 1. : 0.", XMIN=XMIN, XMAX=XMAX, element = Q2.ufl_element())

# **Results**

**Define and solve variational problem:** The variational problem is solved for the Taylor Hood elements and for the equal order elements by formulating the problem on weak residual form with the Galerkin's method where the system is solved for the residual to be zero.

In [ ]:
h = CellDiameter(mesh2)
C = 1.0e3
gamma = C/h

f1 = Expression(("0.0","0.0"), element = V1.ufl_element())
#with equal order
f2 = Expression(("0.0","0.0"), element = V2.ufl_element())

# Define variational problem on residual form: r(u,p;v,q) = 0
residual1 = ( - p1*div(v1)*dx + inner(grad(u1), grad(v1))*dx + div(u1)*q1*dx + 
            gamma*(ib1*inner(u1 - uin, v1) + wb1*inner(u1, v1))*ds - inner(f1, v1)*dx )

# Solve algebraic system 
solve(residual1 == 0, w1) 

#with equal order
# Define variational problem on residual form: r(u,p;v,q) = 0
residual2 = ( - p2*div(v2)*dx + inner(grad(u2), grad(v2))*dx + div(u2)*q2*dx + 
            gamma*(ib2*inner(u2 - uin2, v2) + wb2*inner(u2, v2))*ds - inner(f2, v2)*dx )

# Solve algebraic system 
solve(residual2 == 0, w2) 

**Visualize solution and export files**

**Taylor-Hood Results**

In [ ]:
!rm results-NS/*

# Open files to export solution to Paraview
file_u = File("results-Stokes/u.pvd")
file_p = File("results-Etokes/p.pvd")

u1_1 = project(u1, V1)
p1_1 = project(p1, Q1)

# Save solution to file
file_u << u1_1
file_p << p1_1

# Plot solution
plt.figure()
plot(u1_1, title="Velocity")

plt.figure()
plot(p1_1, title="Pressure")
        
plt.show()

**Equal order Results**

In [ ]:
#with equal order
# Open files to export solution to Paraview
file_u = File("results-Stokes/u.pvd")
file_p = File("results-Etokes/p.pvd")

u1_2 = project(u2, V2)
p1_2 = project(p2, Q2)

# Save solution to file
file_u << u1_2
file_p << p1_2

# Plot solution
plt.figure()
plot(u1_2, title="Velocity")

plt.figure()
plot(p1_2, title="Pressure")
        
plt.show()

# Export files
#!tar -czvf results-Stokes.tar.gz results-NS
#files.download('results-Stokes.tar.gz')

# **Discussion**

With the Taylor Hood elements the solution looks like expected for the velocity and for the pressure. With the equal order elements the solution looks as expected for the velocity but not for the pressure. This is due to the fact that the inf-sup condition is not fulfilled when the order is not sufficiently high for the velocity with respect to the order of the pressure. This gives rise to instabilities in the solution for the pressure which can clearly be seen as the pressure in this case has unphysical attributes. 